# Neural Bandit Algorithm Evaluation Framework

## Graduate Research Project
**AI & Quantum Computing Laboratory**  
**Rochester Institute of Technology**

---

## Research Framework Overview

This comprehensive evaluation framework provides rigorous analysis of neural bandit algorithms with clear categorical distinction between different operational environments:

- **Baseline Environment**: Optimal performance benchmark (Oracle)
- **Stochastic Environment**: Natural random failures and network noise
- **Adversarial Environment**: Strategic intelligent attacks and malicious targeting

## Primary Research Questions

1. **Algorithm Robustness**: How do neural bandit algorithms perform across different threat models?
2. **Comparative Analysis**: Which algorithms demonstrate superior performance in specific scenarios?
3. **Quantified Performance**: What are the exact degradation metrics under adversarial conditions?
4. **Theoretical Validation**: Do experimental results align with established regret bounds?

## Key Research Contributions

- **Systematic Environment Categorization**: Clear baseline/stochastic/adversarial taxonomy
- **Multi-Algorithm Comparative Testing**: Comprehensive evaluation across 6+ algorithms
- **Quantified Robustness Metrics**: Precise performance degradation measurements
- **Publication-Ready Analysis**: Academic-quality visualizations and statistical validation

## Evaluation Methodology

The framework implements standardized testing protocols across three distinct categories:
- **Baseline**: Oracle performance establishing theoretical upper bounds
- **Stochastic**: Random environmental perturbations modeling realistic conditions  
- **Adversarial**: Strategic attack scenarios simulating malicious interference

Each algorithm undergoes identical testing conditions enabling direct performance comparison and robustness quantification across all operational environments.

## Threat Model Classification Framework

### Systematic Environment Taxonomy

This research framework establishes precise categorical distinctions for quantum network evaluation environments, addressing previous ambiguity in threat model classification:

### Environmental Categories

| Environment | Implementation | Threat Characteristic | Research Application |
|-------------|----------------|----------------------|---------------------|
| **Baseline** | `none` | Deterministic optimal performance | Theoretical upper bound |
| **Stochastic** | `stochastic`/`random` | Natural random failures | Realistic network conditions |
| **Adversarial** | `markov` | Oblivious strategic attacks | Pattern-based targeting |
| **Adversarial** | `adaptive` | Responsive strategic attacks | Feedback-driven targeting |
| **Adversarial** | `onlineadaptive` | Real-time strategic attacks | Dynamic threat adaptation |

### Research Contribution

This framework addresses a critical gap in existing literature where random network failures were often conflated with intentional adversarial attacks. The systematic categorization enables:

- **Precise Robustness Quantification**: Exact performance degradation measurements across threat categories
- **Comparative Algorithm Analysis**: Direct performance comparison under identical threat conditions
- **Theoretical Validation**: Empirical verification of regret bounds across different adversarial models
- **Reproducible Research Standards**: Standardized evaluation protocols for quantum network algorithms

### Methodological Significance

Previous research often lacked clear distinction between stochastic and adversarial environments, limiting the ability to assess true algorithm robustness under intentional attacks versus natural network degradation. This framework provides the necessary precision for rigorous academic evaluation of quantum routing algorithms.

## Environment Setup & Library Installation

In [1]:
# ============================================================
# Setup: Quantum MAB Framework (Paper 7 QBGP Testbed)
# ============================================================

# --- Install Dependencies ---
!pip install -q torch torchvision numpy matplotlib seaborn pandas tqdm scipy scikit-learn pmdarima networkx

# --- Core Imports ---
import os, sys, gc, warnings, importlib, subprocess
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import networkx as nx
from pathlib import Path
from tqdm import tqdm

warnings.filterwarnings('ignore')

# --- Path Setup ---
print(f"Current working directory: {os.getcwd().split('/')[-1]}")
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    project_dir = '/content/drive/MyDrive/GA-Work/hybrid_variable_framework/Dynamic_Routing_Eval_Framework'
    os.chdir(project_dir)
    print("Running in Google Colab")
except ImportError:
    print("Running locally (not in Colab)")

sys.path.append(os.path.join(os.getcwd(), 'src'))
print(f"Now working from: {os.getcwd().split('/')[-1]}")

# --- Framework Verification ---
print("Framework dependencies installed successfully")
print(f"Python version: {sys.version.split()[0]}")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"NetworkX version: {nx.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print("Quantum MAB Models Evaluation Framework - Ready for Paper 7 (QBGP) testing")

# --- Cleanup Script Execution ---
root = os.path.abspath("../..")
cleanup_script = os.path.join(root, "cleanup_state_duplicates.py")
if os.path.exists(cleanup_script):
    print(f"\n🚿 Running cleanup script at: {cleanup_script}\n")
    result = subprocess.run(["python3", cleanup_script], text=True, capture_output=True)
    print("===== CLEANUP STDOUT =====\n", result.stdout)
    print("===== CLEANUP STDERR =====\n", result.stderr)
else:
    print("⚠️ Cleanup script not found, skipping...")

# --- Deep Cleanup ---
def deep_cleanup():
    to_clear = ["oracle", "gneuralucb", "expneuralucb", "cpursuitneuralucb",
                "icpursuitneuralucb", "evaluator", "results"]
    for name in to_clear:
        if name in globals():
            obj = globals().get(name)
            if hasattr(obj, "cleanup"): obj.cleanup(verbose=False)
            globals().pop(name, None)
    gc.collect()
    torch.set_default_dtype(torch.float32)
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    print("✓ Deep cleanup complete (memory cleared)")

deep_cleanup()

# Ensure daqr package is discoverable
PARENT_DIR = os.path.abspath("..")
if PARENT_DIR not in sys.path:
    sys.path.insert(0, PARENT_DIR)

# --- Final Module Setup ---
from daqr.core.qubit_allocator              import (QubitAllocator, RandomQubitAllocator, DynamicQubitAllocator, ThompsonSamplingAllocator)
from daqr.core.quantum_physics              import (MemoryNoiseModel, FullPaper2FidelityCalculator, Paper2RewardFunction)
from daqr.evaluation                        import experiment_runner, multi_run_evaluator, visualizer, allocator_runner
from daqr.config                            import experiment_config, gd_backup_manager, local_backup_manager
from daqr.algorithms                        import base_bandit, neural_bandits, predictive_bandits
from daqr.core                              import network_environment, qubit_allocator
from daqr.evaluation.visualizer             import QuantumEvaluatorVisualizer
from daqr.config.gd_backup_manager          import GoogleDriveBackupManager
from daqr.config.experiment_config          import ExperimentConfiguration
from experiments                            import stochastic_evaluation
from daqr.config.local_backup_manager       import LocalBackupManager
from daqr.core.network_environment          import *
from daqr.core.qubit_allocator              import *
from daqr.algorithms.base_bandit            import *
from daqr.algorithms.neural_bandits         import *
from daqr.algorithms.predictive_bandits     import *
from daqr.evaluation.multi_run_evaluator    import *
from daqr.evaluation.experiment_runner      import *

# ============================================================================
# PAPER-SPECIFIC IMPORTS
# ============================================================================
from daqr.core.topology_generator           import Paper2TopologyGenerator
from daqr.core.topology_generator           import Paper7ASTopologyGenerator  # 🆕 Paper 7
from daqr.core.topology_generator           import Paper12WaxmanTopologyGenerator
from daqr.core.quantum_physics              import FiberLossNoiseModel, CascadedFidelityCalculator
from daqr.core.quantum_physics              import FusionNoiseModel, FusionFidelityCalculator, QuARCRewardFunction
from daqr.core.quantum_physics              import Paper12RetryFidelityCalculator
from daqr.core.quantum_physics              import Paper7RewardFunction  # 🆕 Paper 7
from daqr.core                              import attack_strategy

print("✓ All modules reloaded successfully (Paper 7 environment ready)")

# --- Config & Model Setup ---
for module in [experiment_config, network_environment, qubit_allocator, attack_strategy, base_bandit, 
               neural_bandits, predictive_bandits, experiment_runner, multi_run_evaluator, visualizer, 
               stochastic_evaluation]:
    importlib.reload(module)

config = ExperimentConfiguration()
models = config.NEURAL_MODELS

# ============================================================================
# FRAMEWORK CONFIGURATION (Paper 7 Optimized)
# ============================================================================
FRAMEWORK_CONFIG = {
    'exp_num': 5,
    'test_mode': True,
    'base_frames': 4000,
    'frame_step': 2000,
    'models': models,
    'intensity': 0.25,
    'routing_strategy': 'fixed',
    'capacity': 10000,
    'main_env': 'stochastic',

    # Environment parameters
    'env_attrs': {
        'intensity': 0.25,
        'base_seed': 12345,
        'reproducible': True
    },

    'default': {
        'num_paths': 4,
        'total_qubits': 35,
        'min_qubits_per_route': 2,
        'exploration_bonus': 2.0,
        'epsilon': 1.0,
        'seed': 42
    },
    
    # 🎯 Paper #7 (Liu et al. 2024 - QBGP) - PRIMARY TESTBED
    'paper7': {
        # Topology Configuration
        'k': 5,                      # k-shortest paths per ISP pair
        'n_qisps': 3,                # Number of quantum ISP nodes
        'num_paths': 15,              # Total paths for framework compatibility
        'max_nodes': 50,             # AS subgraph size (30-80 for testing, None for full)
        'network_scale': 'small',    # 'small' (30-50), 'medium' (100-200), 'large' (full)
        'topology_path': '/Users/pitergarcia/DataScience/Semester4/GA-Work/hybrid_variable_framework/Dynamic_Routing_Eval_Framework/daqr/core/topology_data/as20000101.txt',
        
        # Framework Parameters
        'total_qubits': 75,
        'min_qubits_per_route': 2,
        'exploration_bonus': 2.0,
        
        # Context & Reward Configuration
        'use_context_rewards': True,      # Enable context-aware reward function
        'reward_mode': 'neg_hop',         # Options: 'neg_hop', 'neg_degree', 'neg_length'
        'use_synthetic': False,           # Force synthetic topology (ignore topology_path)
        
        # Topology Processing
        'largest_cc_only': True,          # Use largest connected component
        'relabel_to_int': True,           # Relabel nodes to integers
        
        # Synthetic Fallback (if topology_path fails or use_synthetic=True)
        'synthetic_kind': 'barabasi_albert',
        'synthetic_params': {
            'n': 50,                      # Number of nodes
            'm': 3                        # Edges per new node
        }
    },
    
    # Paper #2 (Huang et al. - Neural Bandit Work)
    'paper2': {
        # Topology & Paths
        'num_paths': 4,
        'num_total_qubits': 75,
        'dest_node': 14,
        'num_nodes': 15,
        'source_node': 1,
        
        # Base Physics
        'p_init': 0.00001,
        'total_qubits': 75,
        'f_attenuation': 0.05,
        
        # Stochastic Noise Parameters
        'p_BSM': 0.2,
        'p_GateErrors': 0.2,
        'p_depol': 0.1,

        # State Configuration
        'testbed': 'paper2',
        'initial_state': 'idle',
        'state_total_qubits': {'busy': 35, 'idle': 43},
        
        # Bandit Algorithm
        'exploration_bonus': 2.0,
        'min_qubits_per_route': 2,
        'transition_trigger': True,
        'paper2_transition_interval': 50,
        'entanglement_success_factor': 4000,
        
        # Paper2 Features
        'use_paper2_rewards': True,
        'swap_mode': 'async',
        'memory_T2': 5000,
        'gate_error_rate': 0.02,
        'swap_delay_per_link': 100,
        'use_gate_error': True,
        'use_memory_decay': True,
    },
    
    # Paper #12 (Wang et al. 2024 - QuARC)
    'paper12': {
        # Topology
        'n_nodes': 100,
        'avg_degree': 6,
        'waxman_beta': 0.2,
        'waxman_alpha': 0.4,
        'topology_type': 'waxman',
        
        # Physical parameters
        'channel_width': 3,
        'fusion_prob': 0.9,
        'qubits_per_node': 12,
        'entanglement_prob': 0.6,
        
        # Simulation parameters
        'num_sd_pairs': 10,
        'epoch_length': 500,
        'total_timeslots': 7000,
        
        # QuARC-specific
        'split_constant': 4,
        'enable_clustering': True,
        'enable_secondary_fusions': True,
        
        # Framework mapping
        'num_paths': 4,
        'total_qubits': 120,
        'exploration_bonus': 1.5,
        'min_qubits_per_route': 3,
        'use_fusion_rewards': True,

        'time_decay_physics': {
            'memory_lifetime': 0.5
        },

        # Retry parameters
        'retry_threshold': 0.7,
        'max_retry_attempts': 3,
        'retry_decay_rate': 0.95,
        'enable_retry_logging': True,
        'retry_cost_per_attempt': 0.1,
    }
}

# --- Test Scenarios ---
if FRAMEWORK_CONFIG['main_env'] == 'stochastic':
    test_scenarios = {
        'none': 'Baseline (Optimal Conditions)',
        'stochastic': 'Stochastic Random Failures',
        'markov': 'Markov Adversarial Attack',
        'adaptive': 'Adaptive Adversarial Attack',
        'onlineadaptive': 'Online Adaptive Attack'
    }
    evaluation_type = "STOCHASTIC-FOCUSED"
else:
    test_scenarios = {
        'stochastic': 'Stochastic (Natural Network Failures)',
        'adaptive': 'Adversarial (Strategic Attacks)'
    }
    evaluation_type = "COMPARATIVE"

# --- Display Configuration ---
print("=" * 70)
print("DYNAMIC ROUTING EVALUATION FRAMEWORK - PAPER 7 (QBGP) CONFIGURATION")
print("=" * 70)
print(f"Primary Testbed: Paper 7 (Liu et al., 2024) - QBGP Multi-ISP Routing")
print(f"Models to evaluate: {len(models)} total")
print(f"Topology: {'Real AS data' if not FRAMEWORK_CONFIG['paper7']['use_synthetic'] else 'Synthetic Barabási-Albert'}")
print(f"AS Nodes: {FRAMEWORK_CONFIG['paper7']['max_nodes'] or 'Full network'}")
print(f"ISP Nodes: {FRAMEWORK_CONFIG['paper7']['n_qisps']}")
print(f"Paths per ISP pair: {FRAMEWORK_CONFIG['paper7']['k']}-shortest")
print(f"Reward Mode: {FRAMEWORK_CONFIG['paper7']['reward_mode']}")
print(f"Context-aware Rewards: {FRAMEWORK_CONFIG['paper7']['use_context_rewards']}")
print("\n✓ Configuration loaded successfully - Ready for Paper 7 evaluation")
print("=" * 70)

Current working directory: notebooks
Running locally (not in Colab)
Now working from: notebooks
Framework dependencies installed successfully
Python version: 3.12.1
PyTorch version: 2.8.0+cu128
NumPy version: 1.26.4
NetworkX version: 3.5
Matplotlib version: 3.10.6
Quantum MAB Models Evaluation Framework - Ready for Paper 7 (QBGP) testing

🚿 Running cleanup script at: /workspaces/quantum_project/cleanup_state_duplicates.py

===== CLEANUP STDOUT =====
 Script dir: /workspaces/quantum_project
Project root: /workspaces/quantum_project/Dynamic_Routing_Eval_Framework

State roots:
  ✅ /workspaces/quantum_project/Dynamic_Routing_Eval_Framework/daqr/config/framework_state
  ✅ /workspaces/quantum_project/Dynamic_Routing_Eval_Framework/daqr/config/model_state
  ❌ /content/drive/Shareddrives/ai_quantum_computing/quantum_data_lake/framework_state
  ❌ /content/drive/Shareddrives/ai_quantum_computing/quantum_data_lake/model_state

========== STARTING CLEANUP ==========

[CLEANUP] Removing .0 float a

## Comparative Analysis Framework: Stochastic versus Adversarial Environments

### Research Focus

This evaluation constitutes the primary empirical contribution of the research: systematic quantification of algorithm performance across fundamentally different operational conditions that distinguish between natural system failures and intentional strategic attacks.

### Environmental Characterization

**Stochastic Environment**
- **Operational Model**: Natural random failures representing realistic network degradation patterns
- **Attack Distribution**: Probabilistic failures following uniform random distribution
- **Research Significance**: Establishes baseline performance metrics under standard operational conditions

**Adversarial Environment**  
- **Operational Model**: Strategic intelligent attacks systematically targeting algorithmic decision-making processes
- **Attack Distribution**: Adaptive targeting mechanisms that dynamically respond to observed algorithm behavior
- **Research Significance**: Evaluates robustness under worst-case strategic threat scenarios

### Experimental Predictions

Based on the theoretical analysis and algorithm architecture, the following empirical outcomes are anticipated:

**Performance Superiority Hypothesis**
EXPNeuralUCB will demonstrate measurably superior performance retention in adversarial environments relative to baseline neural bandit algorithms lacking specialized adversarial robustness mechanisms.

**Bounded Degradation Hypothesis**
Performance degradation under adversarial conditions will remain within acceptable operational limits, specifically maintaining performance within 85% of stochastic environment baselines.

**Stability Hypothesis**
Algorithm performance rankings will exhibit stability across varying adversarial attack intensities, indicating consistent robustness characteristics rather than scenario-dependent performance fluctuations.

### Research Methodology

The comparative analysis employs identical experimental conditions across both environments, enabling precise quantification of performance degradation attributable to adversarial targeting while controlling for environmental variables and maintaining statistical rigor in the evaluation process.

In [2]:
# ============================================================================
# PAPER 7 HELPER FUNCTIONS
# ============================================================================

def generate_paper7_paths(topology, k: int, n_qisps: int, seed: int):
    """
    Generate k-shortest paths between n_qisps quantum ISP nodes.
    
    Args:
        topology: NetworkX graph (AS-level topology)
        k: Number of shortest paths per ISP pair
        n_qisps: Number of quantum ISP nodes
        seed: Random seed
        
    Returns:
        List of paths (each path is a list of nodes)
    """
    rng = np.random.default_rng(seed)
    nodes = list(topology.nodes())

    if len(nodes) < n_qisps:
        raise ValueError(f"Topology has {len(nodes)} nodes, need {n_qisps} for ISPs")
    
    # Select ISP nodes (prefer high-degree nodes like real BGP)
    degrees = dict(topology.degree())
    sorted_nodes = sorted(nodes, key=lambda n: degrees[n], reverse=True)
    isp_nodes = sorted_nodes[:n_qisps]  # Take top-degree nodes
    
    all_paths = []
    for src, dst in itertools.combinations(isp_nodes, 2):
        try:
            path_generator = nx.shortest_simple_paths(topology, src, dst, weight='distance')
            paths = list(itertools.islice(path_generator, k))
            all_paths.extend(paths)
        except nx.NetworkXNoPath:
            continue
    
    if not all_paths:
        raise RuntimeError(f"Could not find any paths between {n_qisps} ISP nodes")
    
    return all_paths


def generate_paper7_contexts(paths, topology):
    """
    Generate context vectors for each path: [hop_count, avg_degree, path_length].
    
    Args:
        paths: List of paths (each path is a list of nodes)
        topology: NetworkX graph
        
    Returns:
        List of context arrays, one per path (shape: [1, 3])
    """
    contexts = []
    
    for path in paths:
        # Feature 1: Hop count (AS path length)
        hop_count = len(path) - 1
        
        # Feature 2: Average node degree (bottleneck indicator)
        degrees = [topology.degree(node) for node in path]
        avg_degree = sum(degrees) / len(degrees) if degrees else 0.0

        # Feature 3: Physical path length (sum of edge distances)
        path_length = 0.0
        for i in range(len(path) - 1):
            edge_data = topology.get_edge_data(path[i], path[i+1])
            path_length += edge_data.get('distance', 1.0)

        # Context vector: [hop_count, avg_degree, path_length]
        context_vector = np.array([hop_count, avg_degree, path_length], dtype=float)
        contexts.append([context_vector])  # Wrap in list for framework compatibility
    
    return contexts


def get_physics_params(
    physics_model: str = "default",
    current_frames: int = 4000,
    base_seed: int = 42,
    qubit_cap=None,
    *,
    topology: "nx.Graph | None" = None,
    topology_model: str | None = None,
    topology_path: str | Path | None = None,
    topology_max_nodes: int | None = None,
    topology_largest_cc_only: bool = True,
    topology_relabel_to_int: bool = True,
    synthetic_kind: str = "barabasi_albert",
    synthetic_params: dict | None = None,
):
    """
    Unified physics parameter generator for all testbeds.
    
    Returns:
        dict: {noise_model, fidelity_calculator, external_topology, 
               external_contexts, external_rewards}
    """
    
    # ============================================================================
    # 🎯 PAPER 7 (QBGP) - PRIMARY TESTBED
    # ============================================================================
    if physics_model == "paper7":
        import time
        start_time = time.time()
        
        paper7_cfg = FRAMEWORK_CONFIG['paper7']
        node_num = paper7_cfg.get('max_nodes')

        # --- Topology Generation ---
        if topology is not None:
            final_topology = topology
            print(f"📊 Paper7 Topology: User-provided ({len(topology.nodes())} nodes)")
        else:
            # Determine if using synthetic or real AS data
            if paper7_cfg.get('use_synthetic', False) or not paper7_cfg.get('topology_path'):
                # Synthetic fallback
                synth_params = synthetic_params or paper7_cfg.get('synthetic_params', {'n': 50, 'm': 3})
                topo_gen = Paper7ASTopologyGenerator(
                    edge_list_path="dummy_nonexistent.txt",
                    max_nodes=topology_max_nodes or node_num,
                    seed=base_seed,
                    synthetic_fallback=True,
                    synthetic_kind=paper7_cfg.get('synthetic_kind', 'barabasi_albert'),
                    synthetic_params=synth_params
                )
                print(f"📊 Paper7 Topology: Synthetic ({paper7_cfg.get('synthetic_kind')}, n={synth_params.get('n', 50)})")
            else:
                # Real AS topology
                topo_gen = Paper7ASTopologyGenerator(
                    edge_list_path=paper7_cfg['topology_path'],
                    max_nodes=node_num,
                    seed=base_seed,
                    relabel_to_integers=paper7_cfg.get('relabel_to_int', True),
                    largest_cc_only=paper7_cfg.get('largest_cc_only', True),
                    synthetic_fallback=True
                )
                topo_path_short = paper7_cfg['topology_path'].split('/')[-1]
                print(f"📊 Paper7 Topology: Real AS ({topo_path_short})")
            
            final_topology = topo_gen.generate()

        # --- Path Generation ---
        k = paper7_cfg["k"]
        n_qisps = paper7_cfg["n_qisps"]
        paths = generate_paper7_paths(final_topology, k, n_qisps, base_seed)
        contexts = generate_paper7_contexts(paths, final_topology)
        
        elapsed_ms = (time.time() - start_time) * 1000
        print(f"📊 Paper7 Paths: {len(paths)} paths from {k}-shortest between {n_qisps} ISPs")
        print(f"📊 Paper7 Contexts: {len(contexts)} context vectors generated")

        # --- Reward Function (Optional) ---
        external_rewards = None
        if paper7_cfg.get('use_context_rewards', False):
            reward_mode = paper7_cfg.get('reward_mode', 'neg_hop')
            reward_func = Paper7RewardFunction(mode=reward_mode)
            external_rewards = []
            for ctx_list in contexts:
                path_rewards = [reward_func.compute(ctx) for ctx in ctx_list]
                external_rewards.append(path_rewards)
            print(f"📊 Paper7 Rewards: Context-aware (mode={reward_mode})")
        else:
            print(f"📊 Paper7 Rewards: Using default framework rewards")

        print(f"⏱️  get_physics_params_paper7() time: {elapsed_ms:.1f} ms")
        
        return {
            "noise_model": None,
            "fidelity_calculator": None,
            "external_topology": final_topology,
            "external_contexts": contexts,
            "external_rewards": external_rewards
        }
    
    # ============================================================================
    # PAPER 2 (Huang et al.)
    # ============================================================================
    elif physics_model == "paper2":
        p2_config = FRAMEWORK_CONFIG["paper2"]
        topo_gen = Paper2TopologyGenerator(num_nodes=p2_config["num_nodes"], seed=base_seed)
        topo = topo_gen.generate()
        
        try:
            path_generator = nx.shortest_simple_paths(
                topo, p2_config["source_node"], p2_config["dest_node"], weight="distance"
            )
            paths = list(itertools.islice(path_generator, p2_config["num_paths"]))
        except nx.NetworkXNoPath:
            paths = [[p2_config["source_node"], p2_config["dest_node"]]] * p2_config["num_paths"]
        
        # Stochastic noise model
        noise_model = FiberLossNoiseModel(
            topology=topo,
            paths=paths,
            p_init=p2_config.get("p_init", 0.00001),
            f_attenuation=p2_config.get("f_attenuation", 0.05)
        )
        
        # Fidelity calculator with optional memory decay
        if p2_config.get('use_memory_decay', False):
            memory_model = MemoryNoiseModel(
                T2=p2_config.get("memory_T2", 5000),
                swap_delay_per_link=p2_config.get("swap_delay_per_link", 100)
            )
            if p2_config.get("swap_mode", "sync") == "sync":
                memory_model = None
        else:
            memory_model = None
        
        fidelity_calc = FullPaper2FidelityCalculator(
            gate_error_rate=p2_config.get("gate_error_rate", 0.02) if p2_config.get('use_gate_error', False) else 0.0,
            memory_model=memory_model
        )
        
        print(f"📊 Paper2 Physics: Fiber Loss + {'Memory Decay' if memory_model else 'No Memory'}")
        
        return {
            "noise_model": noise_model,
            "fidelity_calculator": fidelity_calc,
            "external_topology": topo,
            "external_contexts": None,
            "external_rewards": None
        }

    # ============================================================================
    # PAPER 12 (Wang et al. - QuARC)
    # ============================================================================
    elif physics_model == 'paper12':
        p12config = FRAMEWORK_CONFIG['paper12']
        
        # Get base Paper12 physics
        physics_params = get_physics_params_paper12(p12config, seed=base_seed, qubit_cap=qubit_cap)
        base_fidelity_calc = physics_params['fidelity_calculator']
        
        # Wrap with retry logic
        fidelity_calc = Paper12RetryFidelityCalculator(
            base_calculator=base_fidelity_calc,
            threshold=p12config['retry_threshold'],
            max_attempts=p12config['max_retry_attempts'],
            decay_rate=p12config['retry_decay_rate']
        )
        
        physics_params['fidelity_calculator'] = fidelity_calc
        
        # Add metadata
        metadata = {
            'paper': 'Wang2024Paper12',
            'retry_enabled': True,
            'retry_threshold': p12config['retry_threshold'],
            'max_attempts': p12config['max_retry_attempts'],
            'decay_rate': p12config['retry_decay_rate'],
        }
        physics_params['metadata'] = metadata
        
        print(f"📊 Paper12 Physics: Fusion (prob={p12config['fusion_prob']}) + Retry Logic")
        
        return physics_params
    
    # ============================================================================
    # DEFAULT (No special physics)
    # ============================================================================
    else:
        return {
            "noise_model": None,
            "fidelity_calculator": None,
            "external_topology": topology,
            "external_contexts": None,
            "external_rewards": None
        }


def get_physics_params_paper12(config, seed, qubit_cap, num_paths = 4):
    """Paper #12 (Waxman + QuARC) physics adapter."""
    topology = Paper12WaxmanTopologyGenerator().generate()
    num_paths = num_paths
    nodes = list(topology.nodes())
    rng = np.random.default_rng(seed)

    # Find 4 paths
    paths = []
    attempts = 0
    max_attempts = 10 * num_paths
    
    while len(paths) < num_paths and attempts < max_attempts:
        attempts += 1
        src, dst = rng.choice(nodes, 2, replace=False)
        try:
            path = nx.shortest_path(topology, src, dst)
            if path not in paths:
                paths.append(path)
        except nx.NetworkXNoPath:
            continue
    
    if len(paths) < num_paths:
        raise RuntimeError(f"Could not find {num_paths} valid paths in Waxman topology")

    # Physics models
    fusion_prob = float(config.get("fusion_prob", 0.9))
    entanglement_prob = float(config.get("entanglement_prob", 0.6))
    noise_model = FusionNoiseModel(
        topology=topology, paths=paths, fusion_prob=fusion_prob, entanglement_prob=entanglement_prob
    )
    fidelity_calc = FusionFidelityCalculator()
    reward_func = QuARCRewardFunction()

    # Contexts: 4 arrays with shapes (8,3), (10,3), (8,3), (9,3)
    external_contexts = []
    arms_per_path = [8, 10, 8, 9]
    degrees = dict(topology.degree())
    max_degree = max(degrees.values()) if degrees else 1.0

    for p_idx, K in enumerate(arms_per_path):
        path = paths[p_idx]
        hop_count = len(path) - 1
        path_degrees = [degrees[n] for n in path]
        avg_degree = float(sum(path_degrees) / len(path_degrees))
        f2_deg_norm = avg_degree / max_degree if max_degree > 0 else 0.0
        ctx = np.full((K, 3), [float(hop_count), f2_deg_norm, fusion_prob], dtype=float)
        external_contexts.append(ctx)

    # Rewards: 4 lists with lengths [8,10,8,9]
    external_rewards = []
    for p_idx, K in enumerate(arms_per_path):
        path = paths[p_idx]
        err_info = noise_model.get_error_rates(p_idx)
        base_fidelity = fidelity_calc.compute_path_fidelity(err_info, context=None, fusion_prob=fusion_prob)
        base_fidelity = float(np.clip(base_fidelity, 0.0, 1.0))

        path_rewards = []
        for _ in range(K):
            success = rng.random() < base_fidelity
            r = reward_func.compute_reward(success=success, aggregate_throughput=1)
            path_rewards.append(float(r))
        
        external_rewards.append(path_rewards)
    
    return {
        "external_topology": topology,
        "external_contexts": external_contexts,
        "external_rewards": external_rewards,
        "noise_model": noise_model,
        "fidelity_calculator": fidelity_calc,
    }


def force_release_resources(evaluator=None, verbose=True):
    """Force release of ALL resources that could block. Call AFTER each allocator completes."""
    cleanup_log = []

    # 1. Stop logging and close file handles
    if evaluator is not None:
        try:
            if hasattr(evaluator, 'configs') and hasattr(evaluator.configs, 'backup_mgr'):
                backup_mgr = evaluator.configs.backup_mgr
                if hasattr(backup_mgr, 'stop_logging_redirect'):
                    backup_mgr.stop_logging_redirect()
                if hasattr(backup_mgr, '_log_file'):
                    try:
                        backup_mgr._log_file.close()
                    except:
                        pass
                if hasattr(backup_mgr, 'backup_registry'):
                    backup_mgr.backup_registry.clear()
            cleanup_log.append("✅ Backup manager cleaned")
        except Exception as e:
            cleanup_log.append(f"⚠️ Backup cleanup: {e}")

    # 2. Clear environment graphs
    if evaluator is not None:
        try:
            if hasattr(evaluator, 'configs') and hasattr(evaluator.configs, 'environment'):
                env = evaluator.configs.environment
                if hasattr(env, 'topology') and hasattr(env.topology, 'clear'):
                    env.topology.clear()
                    del env.topology
                if hasattr(env, 'paths'):
                    env.paths = []
            cleanup_log.append("✅ Environment graphs cleared")
        except Exception as e:
            cleanup_log.append(f"⚠️ Environment cleanup: {e}")

    # 3. Break circular references
    if evaluator is not None:
        try:
            if hasattr(evaluator, 'configs'):
                if hasattr(evaluator.configs, 'backup_mgr'):
                    evaluator.configs.backup_mgr = None
                if hasattr(evaluator.configs, 'environment'):
                    evaluator.configs.environment = None
                evaluator.configs = None
            cleanup_log.append("✅ Circular references broken")
        except Exception as e:
            cleanup_log.append(f"⚠️ Reference cleanup: {e}")

    # 4. Clear model registries
    try:
        import sys
        for mod_name in list(sys.modules.keys()):
            if 'bandit' in mod_name.lower() or 'neural' in mod_name.lower():
                mod = sys.modules[mod_name]
                if hasattr(mod, '_model_registry'):
                    mod._model_registry.clear()
                if hasattr(mod, '_global_models'):
                    mod._global_models.clear()
        cleanup_log.append("✅ Model registries cleared")
    except Exception as e:
        cleanup_log.append(f"⚠️ Registry cleanup: {e}")

    # 5. Torch cleanup
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        cleanup_log.append("✅ Torch CUDA cleared")
    except Exception as e:
        cleanup_log.append(f"⚠️ Torch cleanup: {e}")

    # 6. Garbage collection
    collected = [gc.collect() for _ in range(3)]
    cleanup_log.append(f"✅ GC collected: {sum(collected)} objects")

    # 7. Close file descriptors
    try:
        import psutil
        process = psutil.Process()
        for f in process.open_files():
            if any(ext in f.path for ext in ['.pkl', '.log', '.csv']):
                try:
                    os.close(f.fd)
                except:
                    pass
        cleanup_log.append("✅ File descriptors closed")
    except Exception as e:
        cleanup_log.append(f"⚠️ FD cleanup: {e}")

    # 8. Delete evaluator and final collection
    if evaluator is not None:
        del evaluator
    gc.collect(2)

    if verbose:
        print("\n" + "="*70)
        print("🧹 FORCED RESOURCE RELEASE")
        print("="*70)
        for log in cleanup_log:
            print(log)
        print("="*70 + "\n")

    return True


# ============================================================================
# VALIDATION: Quick Paper 7 Sanity Check
# ============================================================================
print("\n" + "="*70)
print("🔍 PAPER 7 QUICK VALIDATION")
print("="*70)

try:
    # Test Paper 7 physics generation
    test_params = get_physics_params(
        physics_model='paper7',
        base_seed=42
    )
    
    print(f"✅ Topology: {len(test_params['external_topology'].nodes())} nodes, "
          f"{len(test_params['external_topology'].edges())} edges")
    print(f"✅ Contexts: {len(test_params['external_contexts'])} paths")
    print(f"✅ Rewards: {'Enabled' if test_params['external_rewards'] else 'Disabled'}")
    print("\n✓ Paper 7 integration validated successfully")
    
except Exception as e:
    print(f"❌ Validation failed: {e}")
    import traceback
    traceback.print_exc()

print("="*70 + "\n")

print("🚀 Ready to run Paper 7 (QBGP) experiments!")
print("   Example: PHYSICS_MODELS = ['paper7']")
print("            ALLOCATORS = ['ThompsonSampling', 'DynamicUCB']")



🔍 PAPER 7 QUICK VALIDATION
📊 Paper7 Topology: Real AS (as20000101.txt)
📊 Paper7 Paths: 15 paths from 5-shortest between 3 ISPs
📊 Paper7 Contexts: 15 context vectors generated
📊 Paper7 Rewards: Context-aware (mode=neg_hop)
⏱️  get_physics_params_paper7() time: 5.7 ms
✅ Topology: 50 nodes, 141 edges
✅ Contexts: 15 paths
✅ Rewards: Enabled

✓ Paper 7 integration validated successfully

🚀 Ready to run Paper 7 (QBGP) experiments!
   Example: PHYSICS_MODELS = ['paper7']
            ALLOCATORS = ['ThompsonSampling', 'DynamicUCB']


In [3]:
# Verification: Check that fixes are in place
print("=" * 70)
print("VERIFYING NEURAL BANDIT FIXES ARE LOADED")
print("=" * 70)

# Check NeuralUCB clamping fix
from daqr.algorithms.base_bandit import NeuralUCB
import inspect
source = inspect.getsource(NeuralUCB.take_action)
if "np.maximum(p, 0.0)" in source:
    print("✅ NeuralUCB CLAMPING FIX detected")
else:
    print("❌ NeuralUCB CLAMPING FIX NOT found - reload failed!")

# Check EXPNeuralUCB probability fix  
from daqr.algorithms.neural_bandits import EXPNeuralUCB
source = inspect.getsource(EXPNeuralUCB._calculate_group_probabilities)
if "log_sum_exp" in source or "max_exponent" in source:
    print("✅ EXPNeuralUCB PROBABILITY FIX detected")
else:
    print("❌ EXPNeuralUCB PROBABILITY FIX NOT found - reload failed!")

print("=" * 70)

VERIFYING NEURAL BANDIT FIXES ARE LOADED
✅ NeuralUCB CLAMPING FIX detected
✅ EXPNeuralUCB PROBABILITY FIX detected


In [4]:
# Debug cell: Detailed traceback for GNeuralUCB error
print("=" * 70)
print("DETAILED DEBUG: Enhanced error tracing")
print("=" * 70)

import traceback
import sys
import shutil

# Patch numpy.random.choice to catch bad probability arrays
original_choice = np.random.choice

def debug_choice(a, size=None, replace=True, p=None, **kwargs):
    if p is not None:
        p_arr = np.asarray(p)
        if np.any(p_arr < 0):
            print(f"\n  [DEBUG-CHOICE] ❌ CAUGHT BAD PROBABILITIES!")
            print(f"  [DEBUG-CHOICE]    p values: {p_arr}")
            print(f"  [DEBUG-CHOICE]    min: {np.min(p_arr)}, max: {np.max(p_arr)}")
            print(f"  [DEBUG-CHOICE]    sum: {np.sum(p_arr)}")
            print(f"  [DEBUG-CHOICE]    Stack trace:")
            traceback.print_stack()
            raise ValueError("probabilities are not non-negative")
    return original_choice(a, size=size, replace=replace, p=p, **kwargs)

np.random.choice = debug_choice

# Clear cache
print("\n🗑️ CLEARING CACHED MODEL FILES...")
cache_dirs = ['model_state', 'framework_state', 'quantum_logs', '.cache', '__pycache__']
for cache_dir in cache_dirs:
    try:
        if os.path.exists(cache_dir):
            shutil.rmtree(cache_dir)
            print(f"   ✅ Cleared: {cache_dir}")
    except Exception as e:
        print(f"   ⚠️  Could not clear {cache_dir}: {e}")

print("\n✅ Enhanced debugging enabled")


DETAILED DEBUG: Enhanced error tracing

🗑️ CLEARING CACHED MODEL FILES...

✅ Enhanced debugging enabled


In [5]:
# ============================================================
# CRITICAL: Clear All Cached Models Before Running Test
# ============================================================
import os
import shutil

print("=" * 70)
print("🗑️ CLEARING ALL CACHED MODEL FILES")
print("=" * 70)

# Specific cache directories used by the framework
cache_paths = [
    'daqr/config/model_state',
    'daqr/config/framework_state',
    'daqr/config/quantum_logs'
]

for cache_path in cache_paths:
    if os.path.exists(cache_path):
        try:
            shutil.rmtree(cache_path)
            print(f"✅ Cleared: {cache_path}")
        except Exception as e:
            print(f"⚠️  Error clearing {cache_path}: {e}")
    else:
        print(f"ℹ️  Not found: {cache_path}")

print("\n✅ Cache fully cleared - models will be fresh from fixed code")
print("=" * 70)


🗑️ CLEARING ALL CACHED MODEL FILES
ℹ️  Not found: daqr/config/model_state
ℹ️  Not found: daqr/config/framework_state
ℹ️  Not found: daqr/config/quantum_logs

✅ Cache fully cleared - models will be fresh from fixed code


In [6]:
# ============================================================
# Cell 2: Allocator + ExperimentConfiguration for Paper #7
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #7 (QBGP) QUANTUM ROUTING EVALUATION - SINGLE ALLOCATOR TEST")
print("=" * 70)

# ------------------------------------------------------------
# 1) Single Allocator Selection
# ------------------------------------------------------------
allocator_type = "Default"  # Options: "Random", "DynamicUCB", "ThompsonSampling"

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 10
frame_step          = 10
current_experiments = 1
last_backup         = False
base_cap            = False
overwrite           = True  # CRITICAL: Force fresh models, don't load old cached ones

FRAMEWORK_CONFIG['exp_num']        = current_experiments
FRAMEWORK_CONFIG['base_frames']    = current_frames
FRAMEWORK_CONFIG['frame_step']     = frame_step

    # 'exp_num': 5,
    # 'test_mode': True,
    # 'base_frames': 4000,
    # 'frame_step': 2000,
    # 'models': models,
    # 'intensity': 0.25,
    # 'routing_strategy': 'fixed',
    # 'capacity': 10000,
    # 'main_env': 'stochastic',


# Testbed Configuration
PHYSICS_MODELS = ['paper7']  # Paper 7 (QBGP)
ATTACK_SCENARIOS = ['stochastic']  # Start simple
SCALES = [1]
RUNS = [1]

print("\n" + "=" * 70)
print("🎯 PAPER 7 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocator:                  {allocator_type}")
print(f"Physics Model:              {PHYSICS_MODELS[0]}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)

# Run allocator
print(f"\n{'='*70}")
print(f"🚀 RUNNING: {allocator_type} on Paper 7 (QBGP)")
print('='*70)

try:
    # Create isolated runner instance
    custom_config = ExperimentConfiguration(
        env_type=FRAMEWORK_CONFIG['main_env'],
        scenarios=test_scenarios,
        use_last_backup=last_backup,
        models=models,
        attack_intensity=attack_intensity,
        scale=1,
        base_capacity=base_cap,
        overwrite=overwrite
    )

    alloc_runner = AllocatorRunner(
        allocator_type=allocator_type,
        physics_models=PHYSICS_MODELS,
        framework_config=FRAMEWORK_CONFIG,
        scales=SCALES,
        runs=RUNS,
        models=models,
        test_scenarios=test_scenarios,
        config=custom_config
    )

    # Run with Paper 7 physics
    alloc_runner.run(get_physics_params_func=get_physics_params)
    print(f"\n✅ {allocator_type} COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ {allocator_type} FAILED: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 70)
print("✅ ALLOCATOR TEST COMPLETE!")
print("=" * 70)


PAPER #7 (QBGP) QUANTUM ROUTING EVALUATION - SINGLE ALLOCATOR TEST

🎯 PAPER 7 ALLOCATOR EVALUATION
Allocator:                  Default
Physics Model:              paper7
Attack Scenarios:           ['stochastic']
Scales:                     [1]
Runs per Scale:             [1]
Total Frames:               10

🚀 RUNNING: Default on Paper 7 (QBGP)

===================== GET LOCAL REGISTRY =====================
→ Skipping local cache (force=True or file missing)

===================== BUILD LOCAL REGISTRY =====================

🔍 SCANNING LOCAL FILES
Parameters: load_to_drive=False, force=False

📂 Checking DRIVE mode: /content/drive/Shareddrives/ai_quantum_computing/quantum_data_lake
   ⚠️  Path does not exist, skipping

📂 Checking LOCAL mode: /workspaces/quantum_project/Dynamic_Routing_Eval_Framework/daqr/config
   ✅ Path exists, starting walk...
   🚶 Walking directory tree: /workspaces/quantum_project/Dynamic_Routing_Eval_Framework/daqr/config

   📁 Directory: /workspaces/quantum_project/

In [7]:
# ============================================================
# RELOAD FIXED MODULES (Neural Network Probability Fix)
# ============================================================
print("🔄 Reloading fixed neural bandit modules...")

# Reload the fixed modules to apply the probability fixes
importlib.reload(base_bandit)
importlib.reload(neural_bandits)

print("✅ Neural bandit modules reloaded with probability fixes")
print("   - NeuralUCB.take_action() now clamps negative values")
print("   - EXPNeuralUCB._calculate_group_probabilities() is now robust")
print("\n" + "="*70)


🔄 Reloading fixed neural bandit modules...
PyTorch version: 2.8.0+cu128
NumPy version: 1.26.4
Using device: cpu
✅ Neural bandit modules reloaded with probability fixes
   - NeuralUCB.take_action() now clamps negative values
   - EXPNeuralUCB._calculate_group_probabilities() is now robust



In [ ]:
# ============================================================
# Cell 2: Allocator + ExperimentConfiguration for Paper #7
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #7 (QBGP) QUANTUM ROUTING EVALUATION - SINGLE ALLOCATOR TEST")
print("=" * 70)

# ------------------------------------------------------------
# 1) Single Allocator Selection
# ------------------------------------------------------------
allocator_type = "Dynamic"  # Options: "Random", "DynamicUCB", "ThompsonSampling"

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 50
frame_step          = 50
current_experiments = 1
last_backup         = False
base_cap            = False
overwrite           = True

# Testbed Configuration
PHYSICS_MODELS = ['paper7']  # Paper 7 (QBGP)
ATTACK_SCENARIOS = ['stochastic']  # Start simple
SCALES = [1]
RUNS = [5]

print("\n" + "=" * 70)
print("🎯 PAPER 7 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocator:                  {allocator_type}")
print(f"Physics Model:              {PHYSICS_MODELS[0]}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)

# Run allocator
print(f"\n{'='*70}")
print(f"🚀 RUNNING: {allocator_type} on Paper 7 (QBGP)")
print('='*70)

try:
    # Create isolated runner instance
    custom_config = ExperimentConfiguration(
        env_type=FRAMEWORK_CONFIG['main_env'],
        scenarios=test_scenarios,
        use_last_backup=last_backup,
        models=models,
        attack_intensity=attack_intensity,
        scale=1,
        base_capacity=base_cap,
        overwrite=overwrite
    )

    alloc_runner = AllocatorRunner(
        allocator_type=allocator_type,
        physics_models=PHYSICS_MODELS,
        framework_config=FRAMEWORK_CONFIG,
        scales=SCALES,
        runs=RUNS,
        models=models,
        test_scenarios=test_scenarios,
        config=custom_config
    )

    # Run with Paper 7 physics
    alloc_runner.run(get_physics_params_func=get_physics_params)
    print(f"\n✅ {allocator_type} COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ {allocator_type} FAILED: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 70)
print("✅ ALLOCATOR TEST COMPLETE!")
print("=" * 70)


PAPER #7 (QBGP) QUANTUM ROUTING EVALUATION - SINGLE ALLOCATOR TEST

🎯 PAPER 7 ALLOCATOR EVALUATION
Allocator:                  Dynamic
Physics Model:              paper7
Attack Scenarios:           ['stochastic']
Scales:                     [1]
Runs per Scale:             [5]
Total Frames:               50

🚀 RUNNING: Dynamic on Paper 7 (QBGP)

===================== GET LOCAL REGISTRY =====================
→ Skipping local cache (force=True or file missing)

===================== BUILD LOCAL REGISTRY =====================

🔍 SCANNING LOCAL FILES
Parameters: load_to_drive=False, force=False

📂 Checking DRIVE mode: /content/drive/Shareddrives/ai_quantum_computing/quantum_data_lake
   ⚠️  Path does not exist, skipping

📂 Checking LOCAL mode: /workspaces/quantum_project/Dynamic_Routing_Eval_Framework/daqr/config
   ✅ Path exists, starting walk...
   🚶 Walking directory tree: /workspaces/quantum_project/Dynamic_Routing_Eval_Framework/daqr/config

   📁 Directory: /workspaces/quantum_project/

In [ ]:
# ============================================================
# Cell 2: Allocator + ExperimentConfiguration for Paper #2
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.core.qubit_allocator              import (
                                                QubitAllocator,
                                                RandomQubitAllocator,
                                                DynamicQubitAllocator,
                                                ThompsonSamplingAllocator
                                            )

from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration



print("=" * 70, "\nPAPER #2 QUANTUM MAB MODELS EVALUATION FRAMEWORK - STOCHASTIC FOCUS\n", "=" * 70)
# ------------------------------------------------------------
# 1) Allocator selection (Paper #2: start with fixed baseline)
# ------------------------------------------------------------
# You can swap these as needed:
# allocator = RandomQubitAllocator(epsilon=0.5, seed=42) # ============================================================
# Cell 2: Allocator + ExperimentConfiguration for Paper #7
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #7 (QBGP) QUANTUM ROUTING EVALUATION - SINGLE ALLOCATOR TEST")
print("=" * 70)

# ------------------------------------------------------------
# 1) Single Allocator Selection
# ------------------------------------------------------------
allocator_type = "ThompsonSampling"  # Options: "Random", "DynamicUCB", "ThompsonSampling"

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 50
frame_step          = 50
current_experiments = 1
last_backup         = False
base_cap            = False
overwrite           = True

# Testbed Configuration
PHYSICS_MODELS = ['paper7']  # Paper 7 (QBGP)
ATTACK_SCENARIOS = ['stochastic']  # Start simple
SCALES = [1]
RUNS = [5]

print("\n" + "=" * 70)
print("🎯 PAPER 7 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocator:                  {allocator_type}")
print(f"Physics Model:              {PHYSICS_MODELS[0]}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)

# Run allocator
print(f"\n{'='*70}")
print(f"🚀 RUNNING: {allocator_type} on Paper 7 (QBGP)")
print('='*70)

try:
    # Create isolated runner instance
    custom_config = ExperimentConfiguration(
        env_type=FRAMEWORK_CONFIG['main_env'],
        scenarios=test_scenarios,
        use_last_backup=last_backup,
        models=models,
        attack_intensity=attack_intensity,
        scale=1,
        base_capacity=base_cap,
        overwrite=overwrite
    )

    alloc_runner = AllocatorRunner(
        allocator_type=allocator_type,
        physics_models=PHYSICS_MODELS,
        framework_config=FRAMEWORK_CONFIG,
        scales=SCALES,
        runs=RUNS,
        models=models,
        test_scenarios=test_scenarios,
        config=custom_config
    )

    # Run with Paper 7 physics
    alloc_runner.run(get_physics_params_func=get_physics_params)
    print(f"\n✅ {allocator_type} COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ {allocator_type} FAILED: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 70)
print("✅ ALLOCATOR TEST COMPLETE!")
print("=" * 70)
       # 50-50 mix
# allocator = RandomQubitAllocator(epsilon=0.1, seed=42)        # mostly baseline
# allocator = RandomQubitAllocator(epsilon=0.0, seed=42)        # deterministic
# allocator = DynamicQubitAllocator(seed=42)                    # dynamic UCB
# allocator = ThompsonSamplingAllocator(seed=42)                # Thompson

allocator_type =    "ThompsonSampling" # Fixed allocator via environment (Paper #2 baseline)



# ------------------------------------------------------------
# 2) Derive run parameters from FRAMEWORK_CONFIG (Cell 1)
# ------------------------------------------------------------
# current_frames      = FRAMEWORK_CONFIG['base_frames'] 
# frame_step          = FRAMEWORK_CONFIG['frame_step']
# attack_intensity    = FRAMEWORK_CONFIG['intensity']
# current_experiments = FRAMEWORK_CONFIG['exp_num']

attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 50
frame_step          = 50
current_experiments = 1
last_backup         = False
base_cap            = False
overwrite           = True

# PHYSICS_MODELS = ['paper2', 'default']  # Set to ['default', 'paper2'] to test both
ATTACK_SCENARIOS = ['stochastic']  # Start simple, expand later
PHYSICS_MODELS = ['paper2']
# PHYSICS_MODELS = ['default']
ALLOCATORS = ['Default', 'Dynamic', 'ThompsonSampling', 'Random']
SCALES = [2]
RUNS = [5]


print("\n" + "=" * 70, "\n🎯 QUANTUM ROUTING ALLOCATOR EVALUATION\n", "=" * 70)
print(f"Total Allocators to Test:   {len(ALLOCATORS)}")
print(f"Physics Models:             {PHYSICS_MODELS}")
print(f"Allocators:                 {ALLOCATORS}")
print(f"Scales:                     {SCALES}")
print(f"Runs:                       {RUNS}")
print("=" * 70)

# Run each allocator in isolation
print(f"\n{'='*70}")
print(f"🚀 ALLOCATOR {len(ALLOCATORS)}: {allocator_type}")
print('='*70)

try:
    # Create isolated runner instance for this allocator
    custom_config = ExperimentConfiguration(
        env_type=FRAMEWORK_CONFIG['main_env'], scenarios=test_scenarios, use_last_backup=last_backup,
        models=models, attack_intensity=attack_intensity, scale=2, base_capacity=base_cap, overwrite=overwrite)

    alloc_runner = AllocatorRunner(
        allocator_type=allocator_type, physics_models=PHYSICS_MODELS, framework_config=FRAMEWORK_CONFIG, 
        scales=SCALES, runs=RUNS, models=models, test_scenarios=test_scenarios, config=custom_config)

    # Run this allocator with your existing get_physics_params function
    alloc_runner.run(get_physics_params_func=get_physics_params)
    print(f"\n✅ {allocator_type} COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ {allocator_type} FAILED: {e}")
    traceback.print_exc()

print("\n" + "=" * 70)
print("✅ ALL ALLOCATORS COMPLETE!")
print("=" * 70)

In [ ]:
# ============================================================
# Cell 2: Allocator + ExperimentConfiguration for Paper #2
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.core.qubit_allocator              import (
                                                QubitAllocator,
                                                RandomQubitAllocator,
                                                DynamicQubitAllocator,
                                                ThompsonSamplingAllocator
                                            )

from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration



print("=" * 70, "\nPAPER #2 QUANTUM MAB MODELS EVALUATION FRAMEWORK - STOCHASTIC FOCUS\n", "=" * 70)
# ------------------------------------------------------------
# 1) Allocator selection (Paper #2: start with fixed baseline)
# ------------------------------------------------------------
# You can swap these as needed:
# allocator = RandomQubitAllocator(epsilon=0.5, seed=42) # ============================================================
# Cell 2: Allocator + ExperimentConfiguration for Paper #7
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #7 (QBGP) QUANTUM ROUTING EVALUATION - SINGLE ALLOCATOR TEST")
print("=" * 70)

# ------------------------------------------------------------
# 1) Single Allocator Selection
# ------------------------------------------------------------
allocator_type = "Random"  # Options: "Random", "DynamicUCB", "ThompsonSampling"

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 50
frame_step          = 50
current_experiments = 1
last_backup         = False
base_cap            = False
overwrite           = True

# Testbed Configuration
PHYSICS_MODELS = ['paper7']  # Paper 7 (QBGP)
ATTACK_SCENARIOS = ['stochastic']  # Start simple
SCALES = [1]
RUNS = [5]

print("\n" + "=" * 70)
print("🎯 PAPER 7 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocator:                  {allocator_type}")
print(f"Physics Model:              {PHYSICS_MODELS[0]}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)

# Run allocator
print(f"\n{'='*70}")
print(f"🚀 RUNNING: {allocator_type} on Paper 7 (QBGP)")
print('='*70)

try:
    # Create isolated runner instance
    custom_config = ExperimentConfiguration(
        env_type=FRAMEWORK_CONFIG['main_env'],
        scenarios=test_scenarios,
        use_last_backup=last_backup,
        models=models,
        attack_intensity=attack_intensity,
        scale=1,
        base_capacity=base_cap,
        overwrite=overwrite
    )

    alloc_runner = AllocatorRunner(
        allocator_type=allocator_type,
        physics_models=PHYSICS_MODELS,
        framework_config=FRAMEWORK_CONFIG,
        scales=SCALES,
        runs=RUNS,
        models=models,
        test_scenarios=test_scenarios,
        config=custom_config
    )

    # Run with Paper 7 physics
    alloc_runner.run(get_physics_params_func=get_physics_params)
    print(f"\n✅ {allocator_type} COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ {allocator_type} FAILED: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 70)
print("✅ ALLOCATOR TEST COMPLETE!")
print("=" * 70)
       # 50-50 mix
# allocator = RandomQubitAllocator(epsilon=0.1, seed=42)        # mostly baseline
# allocator = RandomQubitAllocator(epsilon=0.0, seed=42)        # deterministic
# allocator = DynamicQubitAllocator(seed=42)                    # dynamic UCB
# allocator = ThompsonSamplingAllocator(seed=42)                # Thompson

allocator_type =    "ThompsonSampling" # Fixed allocator via environment (Paper #2 baseline)



# ------------------------------------------------------------
# 2) Derive run parameters from FRAMEWORK_CONFIG (Cell 1)
# ------------------------------------------------------------
# current_frames      = FRAMEWORK_CONFIG['base_frames'] 
# frame_step          = FRAMEWORK_CONFIG['frame_step']
# attack_intensity    = FRAMEWORK_CONFIG['intensity']
# current_experiments = FRAMEWORK_CONFIG['exp_num']

attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 50
frame_step          = 50
current_experiments = 1
last_backup         = False
base_cap            = False
overwrite           = True

# PHYSICS_MODELS = ['paper2', 'default']  # Set to ['default', 'paper2'] to test both
ATTACK_SCENARIOS = ['stochastic']  # Start simple, expand later
PHYSICS_MODELS = ['paper2']
# PHYSICS_MODELS = ['default']
ALLOCATORS = ['Default', 'Dynamic', 'ThompsonSampling', 'Random']
SCALES = [2]
RUNS = [5]


print("\n" + "=" * 70, "\n🎯 QUANTUM ROUTING ALLOCATOR EVALUATION\n", "=" * 70)
print(f"Total Allocators to Test:   {len(ALLOCATORS)}")
print(f"Physics Models:             {PHYSICS_MODELS}")
print(f"Allocators:                 {ALLOCATORS}")
print(f"Scales:                     {SCALES}")
print(f"Runs:                       {RUNS}")
print("=" * 70)

# Run each allocator in isolation
print(f"\n{'='*70}")
print(f"🚀 ALLOCATOR {len(ALLOCATORS)}: {allocator_type}")
print('='*70)

try:
    # Create isolated runner instance for this allocator
    custom_config = ExperimentConfiguration(
        env_type=FRAMEWORK_CONFIG['main_env'], scenarios=test_scenarios, use_last_backup=last_backup,
        models=models, attack_intensity=attack_intensity, scale=2, base_capacity=base_cap, overwrite=overwrite)

    alloc_runner = AllocatorRunner(
        allocator_type=allocator_type, physics_models=PHYSICS_MODELS, framework_config=FRAMEWORK_CONFIG, 
        scales=SCALES, runs=RUNS, models=models, test_scenarios=test_scenarios, config=custom_config)

    # Run this allocator with your existing get_physics_params function
    alloc_runner.run(get_physics_params_func=get_physics_params)
    print(f"\n✅ {allocator_type} COMPLETED SUCCESSFULLY")

except Exception as e:
    print(f"\n❌ {allocator_type} FAILED: {e}")
    traceback.print_exc()

print("\n" + "=" * 70)
print("✅ ALL ALLOCATORS COMPLETE!")
print("=" * 70)

## Multi-Environment Performance Analysis

### Complete Evaluation Matrix

This research extends beyond the primary stochastic-adversarial comparison to provide comprehensive algorithm assessment across the complete spectrum of operational environments, establishing a thorough empirical foundation for robustness evaluation.

### Environmental Test Framework

| Environment | Classification | Threat Characteristics | Analytical Purpose |
|-------------|---------------|----------------------|-------------------|
| `none` | Baseline | Deterministic optimal conditions | Theoretical performance ceiling |
| `stochastic` | Probabilistic | Uniform random failures | Standard operational baseline |
| `markov` | Adversarial | Memory-dependent strategic attacks | Oblivious adversarial model |
| `adaptive` | Adversarial | Feedback-driven strategic attacks | Responsive adversarial model |
| `onlineadaptive` | Adversarial | Real-time adaptive strategic attacks | Sophisticated adversarial model |

### Research Contributions

**Comprehensive Threat Model Coverage**
The evaluation framework addresses the complete spectrum of operational conditions, from optimal deterministic environments through increasingly sophisticated adversarial scenarios, providing unprecedented coverage of realistic deployment conditions.

**Graduated Adversarial Complexity Analysis**  
The systematic progression from oblivious to sophisticated adversarial models enables precise quantification of algorithm performance degradation as threat sophistication increases, revealing critical robustness thresholds.

**Cross-Environment Validation Protocol**
Consistent algorithm ranking across multiple environments validates robustness claims and identifies algorithms with stable performance characteristics independent of operational conditions.

**Empirical Robustness Quantification**
The multi-environment approach enables precise measurement of performance degradation rates, establishing quantitative robustness metrics that support theoretical predictions and practical deployment decisions.

### Methodological Significance

This comprehensive evaluation protocol addresses limitations in existing literature where algorithm assessment often focuses on narrow operational scenarios, providing the empirical foundation necessary for robust algorithm deployment in practical quantum network environments.